# Overview

This notebook contains three sections of code which can be used to do the following tasks:
1. Use the OpenAI API to convert Markdown text into structured JSON
2. Convert the output of the above task (a JSON file) into a CSV file for further processing
3. Use the OpenAI API to convert Markdown text directory into a CSV/xlsx spreadsheet file

These tasks are part of a data processing pipeline for transforming historic texts into machine-readable data. To read more about how this pipeline was used in practice, see our datasets created from the [1871 Rivers Pollution Committee Report]('https://github.com/congruence-engine/connecting-environmental-data/tree/main/datasets/1871%20Rivers%20Pollution%20Commission'). 

These code snippets were created with the help of Chat GPT.

1. Markdown to JSON

In [ ]:
import os
import json
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
client = OpenAI()

In [ ]:
def process_file(client, input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as file:
        text_content = file.read()

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant with excellent skills in parsing structured data."},
                {"role": "user", "content": f"Output information in JSON format according to the response format, using the markdown text supplied. Each entry should have three fields: 'business_name', 'business_type', 'address', and 'reply'. 'reply' should include the full text shown under each business - do not leave anything out. DO NOT include additional fields. DO NOT return in a codeblock. Do not include any explanations or extra text. Ensure that the JSON is properly formatted and valid. Do not include any comments or notes. Markdown Text: {text_content}"}
            ],
            max_tokens=2048,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "businesses_info",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "businesses": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "business_name": {"type": "string"},
                                        "business_type": {"type": "string"},
                                        "address": {"type": "string"},
                                        "reply": {"type": "string"}
                                    },
                                    "required": ["business_name", "business_type", "address", "reply"],
                                    "additionalProperties": False
                                }
                            }
                        },
                        "required": ["businesses"],
                        "additionalProperties": False
                    },
                    "strict": True
                }
            }
        )

        with open(output_path, 'w', encoding='utf-8') as json_file:
            json.dump(json.loads(completion.choices[0].message.content), json_file, indent=2)

        return f"Processed {os.path.basename(input_path)} -> {os.path.basename(output_path)}"

    except Exception as e:
        return f"Error processing {os.path.basename(input_path)}: {str(e)}"

def process_txt_files(input_dir, output_dir, max_workers=5):
    client = OpenAI()

    os.makedirs(output_dir, exist_ok=True)

    tasks = []
    for filename in os.listdir(input_dir):
        if filename.endswith('.txt'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.json")
            tasks.append((input_path, output_path))

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_file, client, input_path, output_path): input_path 
                          for input_path, output_path in tasks}
        
        for future in as_completed(future_to_file):
            print(future.result())

# Usage
input_directory = 'input/directory' # Replace with your inpput directory
output_directory = 'output/directory' # Replace with your output directory

process_txt_files(input_directory, output_directory, max_workers=5)

2. JSON to CSV

In [ ]:
import json
import csv

def json_to_csv(json_file_path, output_csv):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    businesses = data.get('businesses', [])

    if businesses:
        fieldnames = businesses[0].keys()

        with open(output_csv, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for business in businesses:
                writer.writerow(business)

        print(f"CSV file '{output_csv}' has been created successfully.")
    else:
        print("No businesses found in the JSON file.")

json_file_path = 'json/file/path/file.json'  # Replace with your JSON file path
output_csv = 'output/file/path/file.csv'    # Replace with your CSV file path

json_to_csv(json_file_path, output_csv)

3. Markdown to CSV/xlsx

In [ ]:
import os
import pandas as pd
import openai
from io import StringIO


def extract_information(text):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant with excellent skills in parsing structured data."},
            {"role": "user", "content": f"From the markdown text provided, output the text in csv format with three columns: 'company_name', 'address', and 'answers_to_questions'. DO NOT include additional columns. Use pipes (|) as separators instead of commas. DO NOT return in a codeblock. Just return the text in csv format, ensuring that no text except column headers is left out. Skip any pipes (|) that appear in the source text. Markdown Text: {text}"}
        ]
    )
    
    response = completion.choices[0].message.content.strip()
    return response

df = pd.DataFrame(columns=['company_name', 'address', 'answers_to_questions'])

input_directory = 'inoput/directory'  # Replace with your directory path

for filename in sorted(os.listdir(input_directory)):
    if filename.endswith('.txt'):
        filepath = os.path.join(input_directory, filename)
        print(f"Processing file: {filename}")
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        
        try:
            csv_text = extract_information(text)
            temp_df = pd.read_csv(StringIO(csv_text), sep='|')
            df = pd.concat([df, temp_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            continue



output_excel_path = 'output/path/file.xlsx'  # Replace with your output path. In this case we have used .xlsx
df.to_excel(output_excel_path, index=False)

print(f"Data has been successfully written to '{output_excel_path}'")